In [40]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [41]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
243,I love Umberto Lenzi's cop movies -- ROME ARME...,negative
955,"Oh my lord, what were they thinking about with...",negative
596,Emma is a horribly flawed film based on Jane A...,positive
777,Over the years I've watched this movie many ti...,positive
709,I don't understand the low 5.7 rating on this ...,positive


In [42]:
import nltk
nltk.download('wordnet')     # For lemmatizer
nltk.download('omw-1.4')     # Optional, but helps with WordNet
nltk.download('stopwords')   # Since you're using stopwords too

# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

# applying all of these functions to our data --> df 
def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/manavdesai/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/manavdesai/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/manavdesai/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [43]:
df = normalize_text(df)
df.head()

,review,sentiment
243,love umberto lenzi s cop movie rome armed teet...,negative
955,oh lord thinking one frantically unfunny worse...,negative
596,emma horribly flawed film based jane austen cl...,positive
777,year watched movie many time seeing hbo copy d...,positive
709,understand low rating film delight people like...,positive


In [44]:
df['sentiment'].value_counts()

sentiment
negative    257
positive    243
Name: count, dtype: int64

In [45]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]
# it will remove the neutral reviews or any other reviews

In [46]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head(5)

,review,sentiment
243,love umberto lenzi s cop movie rome armed teet...,0
955,oh lord thinking one frantically unfunny worse...,0
596,emma horribly flawed film based jane austen cl...,1
777,year watched movie many time seeing hbo copy d...,1
709,understand low rating film delight people like...,1


In [47]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [65]:
vectorizer = CountVectorizer(max_features=50)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [69]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [70]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/Manav54321/MLOps_capstone.mlflow')
dagshub.init(repo_owner='Manav54321', repo_name='MLOps_capstone', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


Initialized MLflow to track repo "Manav54321/MLOps_capstone"

2025-04-19 15:27:38,036 - INFO - Initialized MLflow to track repo "Manav54321/MLOps_capstone"


Repository Manav54321/MLOps_capstone initialized!

2025-04-19 15:27:38,038 - INFO - Repository Manav54321/MLOps_capstone initialized!


<Experiment: artifact_location='mlflow-artifacts:/be1f6338b3b3468eb1da856dd2266a09', creation_time=1744878744632, experiment_id='0', last_update_time=1744878744632, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [71]:
import mlflow
import logging
import os
import time 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 50)
        mlflow.log_param("test_size", 0.20)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-04-19 15:27:42,124 - INFO - Starting MLflow run...
2025-04-19 15:27:42,670 - INFO - Logging preprocessing parameters...
2025-04-19 15:27:45,303 - INFO - Initializing Logistic Regression model...
2025-04-19 15:27:45,307 - INFO - Fitting the model...
2025-04-19 15:27:45,326 - INFO - Model training complete.
2025-04-19 15:27:45,327 - INFO - Logging model parameters...
2025-04-19 15:27:46,018 - INFO - Making predictions...
2025-04-19 15:27:46,021 - INFO - Calculating evaluation metrics...
2025-04-19 15:27:46,031 - INFO - Logging evaluation metrics...
2025-04-19 15:27:48,341 - INFO - Saving and logging the model...
2025-04-19 15:27:48,352 - ERROR - An error occurred: cannot import name '_load_context_model_and_signature' from 'mlflow.pyfunc.model' (/opt/anaconda3/envs/atlas/lib/python3.10/site-packages/mlflow/pyfunc/model.py)
Traceback (most recent call last):
  File "/var/folders/96/8d_2l3_j2vn7zbh4q2tkhxbm0000gn/T/ipykernel_59370/3375003421.py", line 48, in <module>
    mlflow.sklear

🏃 View run placid-ram-832 at: https://dagshub.com/Manav54321/MLOps_capstone.mlflow/#/experiments/0/runs/9985d1c1eb5b4683b0f550949f7331ff
🧪 View experiment at: https://dagshub.com/Manav54321/MLOps_capstone.mlflow/#/experiments/0
